In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import numpy as np
import time
import csv
from Dispensing_wrapper_robot import *
from scale_driver import *
from motor_driver import *


In [43]:
coms = {
    'scaleCom': 'COM7',
    'motorsCom': 'COM9',
}

#Robot Connection
robot = Robot()
robot.initialize()
robot.GoTo_InitialPoint()

ROBOT_IP: 192.168.1.200, VERSION: v2.3.0, PROTOCOL: V1, DETAIL: 6,6,XI1303,MC1303,v2.3.0, TYPE1300: [1, 1]
change protocol identifier to 3
[motion_enable], xArm is not ready to move
[set_state], xArm is ready to move


In [ ]:
robot.GoTo_InitialPoint()

[SDK][ERROR][2024-05-28 13:55:20][base.py:374] - - API -> set_position -> code=9, pos=[-228.0, 0.0, 133.0, 0.0, 1.5707963267948966, 3.141592653589793], radius=-1, velo=20.0, acc=2000


[SDK][ERROR][2024-05-28 13:55:54][base.py:293] - - [main-socket] recv error: [WinError 10054] An existing connection was forcibly closed by the remote host
[SDK][ERROR][2024-05-28 13:55:54][base.py:247] - - [report-socket] recv error: [WinError 10054] An existing connection was forcibly closed by the remote host


In [44]:
#Motor Connection
motors = SerialConnection(coms['motorsCom'], 9600, 10)
dispenser_motor = Motor(motors, 0)

#Scale Connection
scale = Scale(coms['scaleCom'], 9600, 10)
#scale.connect()
#scale.tare()

In [ ]:
scale.tare()

In [52]:

scale.close()

In [53]:
del scale

In [54]:
scale = Scale(coms['scaleCom'], 9600, 10)


In [55]:
scale.connect()

In [41]:
# Define your parameters
acceleration = 10
speed = 5
material = "SuperP"
version_inside = "Design4"
version_outside = "Design1"

In [20]:
dispenser_motor.move(1000, True) #0.0063g per 200
#0,0086 per 400
#0,0026 per 200
#0,0083 per 200

#0,0167 per 1000
#0,0172 per 1000
#0,0132 per 1000


In [45]:
#Error test: Test with Super P and Design 4, see at which point the error does not change anymore significantly 
#Motor is doing 200steps for one full turn, Disgn4 has 12 pits which means 200/12 = 16.6 steps are neccesary to empty one pit

calibration = Calibration(acceleration, speed, material, version_inside, version_outside, dispenser_motor, scale, robot)

#calibration steps = 200/n, 2x200/n, 3x200/n, 4x200/n, 5x200/n, 6x200/n, 7x200/n, 8x200/n, 9x200/n, 10x200/n, 11x200/n, 12x/200, 200
#Design = 12holes
#calibration_steps = [17, 34, 51, 68, 85, 102, 200, 400, 600, 800, 1000, 2000]
calibration_steps = [17, 33, 50, 67, 83, 100, 200, 1000, 2000, 3000, 4000]
calibration_repeat = 5

calibration.calibrate(calibration_steps, calibration_repeat, "Vial1")

calibration.save_calibration()

Repeat: 1
Step: 17, Weight: -0.0001
Scale closed


AttributeError: 'Scale' object has no attribute '_serial_args'

In [22]:
calibration_steps = [12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 132, 144, 200, 400, 600, 800, 1000, 2000]
calibration_repeat = 10

material_per_step = 0.00005
total_steps = sum(calibration_steps)*calibration_repeat
total_material= sum(step*material_per_step for step in calibration_steps)*calibration_repeat
total_material
total_steps

59360